In [19]:
import numpy as np
import xarray as xr
import uwacan

In [15]:
recorder = uwacan.recordings.SoundTrap.read_folder(
    r"D:\Tern Island Vinga\Autonomous Recorders\IVL SoundTrap 1",
    sensor=uwacan.sensor("ST1", sensitivity=-180),
)

In [3]:
for file in recorder.files:
    print(file.start_time, file.stop_time)

2023-08-30 01:28:26+00:00 2023-08-30 04:28:25.411333+00:00
2023-08-30 04:28:24+00:00 2023-08-30 07:28:23.530167+00:00
2023-08-30 07:28:23+00:00 2023-08-30 10:28:22.424833+00:00
2023-08-30 10:28:22+00:00 2023-08-30 13:28:21.431333+00:00
2023-08-30 13:28:21+00:00 2023-08-30 16:28:20.456167+00:00
2023-08-30 16:28:20+00:00 2023-08-30 19:28:19.526667+00:00
2023-08-30 19:28:19+00:00 2023-08-30 22:28:18.481167+00:00
2023-08-30 22:28:18+00:00 2023-08-31 01:28:17.218500+00:00
2023-09-06 01:27:13+00:00 2023-09-06 04:27:12.352667+00:00
2023-09-06 04:27:11+00:00 2023-09-06 07:27:10.364500+00:00
2023-09-06 07:27:10+00:00 2023-09-06 10:27:09.308167+00:00
2023-09-06 10:27:09+00:00 2023-09-06 13:27:08.288000+00:00
2023-09-06 13:27:08+00:00 2023-09-06 16:27:07.181667+00:00
2023-09-06 16:27:06+00:00 2023-09-06 19:27:05.288333+00:00
2023-09-06 19:27:05+00:00 2023-09-06 22:27:04.274000+00:00
2023-09-06 22:27:04+00:00 2023-09-07 01:27:03.494167+00:00


In [37]:
self = recorder.subwindow(center="2023-08-30 13:28:20", duration=120)  # Includes a file boundary
self = recorder.subwindow(center="2023-08-30 12:00:00", duration=120)  # Does not include a file boundary
framesize = 10  # In seconds
overlap = -1  # Fraction
copy_on_out = True

# def rolling(...)
samplerate = self.samplerate
start_time = self.time_window.start
stop_time = self.time_window.stop

# Convert times to samples
samples_to_read = (stop_time - start_time).total_seconds() * samplerate
overlap = round(framesize * samplerate * overlap)
framesize = round(framesize * samplerate)
reuse = max(0, overlap)
num_frames = int((samples_to_read - overlap) / (framesize - overlap))

buffer = np.zeros((framesize, self.num_channels)).squeeze()
if reuse:
    buffer[framesize - reuse:] = self.raw_data(start_offset=0, samples_to_read=reuse)

frames = []
for frame_idx in range(num_frames):
    buffer[:reuse] = buffer[framesize-reuse:]
    start_idx = frame_idx * (framesize - overlap)
    buffer[reuse:] = self.raw_data(start_offset=start_idx + reuse, samples_to_read=framesize - reuse)
    out = buffer.copy() if copy_on_out else buffer
    frames.append(out)
    # yield out

In [5]:
single_frame = self.raw_data_old()
for frame_idx, frame in enumerate(frames):
    start_idx = frame_idx * (framesize - overlap)
    print(np.allclose(frame, single_frame[start_idx:start_idx + framesize]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True


np.float64(999999999.9999999)

In [46]:
def rolling_base(self, framesize, overlap, copy_on_out=False):
    samplerate = self.samplerate
    start_time = self.time_window.start
    stop_time = self.time_window.stop

    # Convert times to samples
    samples_to_read = (stop_time - start_time).total_seconds() * samplerate
    overlap = round(framesize * samplerate * overlap)
    framesize = round(framesize * samplerate)
    reuse = max(0, overlap)
    num_frames = int((samples_to_read - overlap) / (framesize - overlap))

    buffer = np.zeros((framesize, self.num_channels)).squeeze()
    if reuse:
        buffer[framesize - reuse:] = self.raw_data(reference_time=start_time, start_offset=0, samples_to_read=reuse)

    # We want to apply a manual calibration to the raw data, before
    # we stick it in the container. We want to get the calibration scaling
    # in the correct order, corresponding to the sensor order in
    # the raw data. We can do this by "calibrating" a 1.
    # To make sure that the calibration values actually line up with
    # the order of the data, we can get time data with length 0
    # and ask xarray to align the calibrations to this.
    # calibration = uwacan.recordings.calibrate_raw_data(
    #     1,
    #     gain=self.gain,
    #     sensitivity=self.sensor.get("sensitivity"),
    #     adc_range=self.adc_range,
    #     file_range=self.file_range,
    # )
    # dummy_data = self.subwindow(start=True, duration=0).time_data().data
    # calibration = xr.align(dummy_data, calibration)[1].data

    for frame_idx in range(num_frames):
        buffer[:reuse] = buffer[framesize-reuse:]
        start_idx = frame_idx * (framesize - overlap)
        buffer[reuse:] = self.raw_data(reference_time=start_time, start_offset=start_idx + reuse, samples_to_read=framesize - reuse)
        out = buffer.copy() if copy_on_out else buffer
        yield out

def rolling(self, framesize, overlap, return_numpy=False):
    calibration = uwacan.recordings.calibrate_raw_data(
        1,
        gain=self.gain,
        sensitivity=self.sensor.get("sensitivity"),
        adc_range=self.adc_range,
        file_range=self.file_range,
    )
    dummy_data = self.subwindow(start=True, duration=0).time_data().data
    calibration = xr.align(dummy_data, calibration)[1].data

    start_time = self.time_window.start

    for frame_idx, frame in enumerate(rolling_base(self, framesize=framesize, overlap=overlap)):
        frame = frame * calibration
        if return_numpy:
            yield frame
        else:
            yield uwacan._core.TimeData(
                frame,
                samplerate=samplerate,
                start_time=start_time.add(seconds=frame_idx * framesize * (1 - overlap)),
                coords=dummy_data.coords,
                dims=dummy_data.dims,
            )

In [49]:
self = recorder.subwindow(center="2023-08-30 13:28:20", duration=120)  # Includes a file boundary
# self = recorder.subwindow(center="2023-08-30 12:00:00", duration=120)  # Does not include a file boundary
framesize = 10  # In seconds
overlap = 0.5 # Fraction

single_frame = self.time_data()
for frame_idx, frame in enumerate(rolling(self, framesize=framesize, overlap=overlap)):
    frame_time_offset = frame_idx * (framesize - framesize * overlap)
    frame_time_start = self.time_window.start.add(seconds=frame_time_offset)
    true = single_frame.subwindow(start=frame_time_start, duration=framesize)
    print(np.max(np.abs(frame - true)).data)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [33]:
frame.time_window

TimeWindow(start=2023-08-30 13:29:04+00:00, stop=2023-08-30 13:29:14+00:00)

In [34]:
true.time_window

TimeWindow(start=2023-08-30 13:29:04+00:00, stop=2023-08-30 13:29:14+00:00)

In [5]:
file_data = np.arange(200)

# frame_size = 20
# overlap = 5
def get_data(start, stop, verbose=True):
    if verbose:
        print(f"Getting data {start} to {stop}")
    return np.arange(start, stop)

def rolling(total_size, framesize, overlap, copy_on_out=False):
    buffer = np.zeros(shape=framesize)

    num_frames = int((total_size - overlap) / (framesize - overlap))
    for frame_idx in range(num_frames):
        start_idx = frame_idx * (framesize - overlap)
        # frame = x[start_idx:start_idx + framesize]
        buffer[:] = get_data(start_idx, start_idx + framesize)
        # buffer[:] = x[start_idx:start_idx + framesize]
        out = buffer.copy() if copy_on_out else buffer
        yield out

def rolling(total_size, framesize, overlap, copy_on_out=False):
    buffer = np.zeros(shape=framesize)
    num_frames = int((total_size - overlap) / (framesize - overlap))

    if overlap:
        buffer[-overlap:] = get_data(0, overlap)

    for frame_idx in range(num_frames):
        buffer[:overlap] = buffer[-overlap:]
        start_idx = frame_idx * (framesize - overlap)
        buffer[overlap:] = get_data(start_idx + overlap, start_idx + framesize)
        out = buffer.copy() if copy_on_out else buffer
        yield out

def rolling(total_size, framesize, overlap, copy_on_out=False):
    buffer = np.zeros(shape=framesize)
    num_frames = int((total_size - overlap) / (framesize - overlap))
    reuse = max(overlap, 0)

    if reuse:
        buffer[framesize - reuse:] = get_data(0, reuse)

    for frame_idx in range(num_frames):
        buffer[:reuse] = buffer[framesize-reuse:]
        start_idx = frame_idx * (framesize - overlap)
        buffer[reuse:] = get_data(start_idx + reuse, start_idx + framesize)
        out = buffer.copy() if copy_on_out else buffer
        yield out


frames = rolling(200, 20, -5)
for frame in frames:
    print(frame)


Getting data 0 to 20
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
Getting data 25 to 45
[25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40. 41. 42.
 43. 44.]
Getting data 50 to 70
[50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.
 68. 69.]
Getting data 75 to 95
[75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. 89. 90. 91. 92.
 93. 94.]
Getting data 100 to 120
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119.]
Getting data 125 to 145
[125. 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138.
 139. 140. 141. 142. 143. 144.]
Getting data 150 to 170
[150. 151. 152. 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163.
 164. 165. 166. 167. 168. 169.]
Getting data 175 to 195
[175. 176. 177. 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188.
 189. 190. 191. 192. 193. 194.]
